In [1]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, RepeatedKFold, GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LinearRegression, Lasso, Ridge, RidgeCV, ElasticNet, ElasticNetCV, MultiTaskElasticNet
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
import pandas as pd
import numpy as np
import matplotlib as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error

from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.manifold import TSNE

In [10]:
# acled_df = pd.read_csv('../data/acled/south_sudan.csv') 
# # acled_df = acled_df.drop(['SOURCE', 'SOURCE_SCALE', 'TAGS', 'EVENT_DATE', 'YEAR', 'GEO_PRECISION'], axis=1)
# data = acled_df
# X = data.drop('FATALITIES', axis=1)
# y = data['FATALITIES']

ucdp_df = pd.read_csv("../data/ucdp/darfur_113023.csv").drop('low_est', axis=1) ## 1669 entries, 48 cols
ucdp_df = ucdp_df.drop(['deaths_a', 'deaths_b', 'deaths_civilians', 'deaths_unknown', 'high_est', 'number_of_sources', 'source_date', 'source_office', 'source_headline', 'source_article', 'source_original'], axis=1)

# print(ucdp_df.info()) 
print(ucdp_df.columns)
ucdp_df.dropna(inplace=True)

data = ucdp_df
X = data.drop('best_est', axis=1)
y = data['best_est']


categorical_features = X.select_dtypes(include=['object']).columns.tolist()
numeric_features = X.select_dtypes(include=['int64', 'float64']).columns.tolist()

numeric_transformer = StandardScaler()
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

# Create a column transformer to apply different preprocessing to different columns
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

X = preprocessor.fit_transform(X)

Index(['id', 'relid', 'year', 'active_year', 'code_status', 'type_of_violence',
       'conflict_dset_id', 'conflict_new_id', 'conflict_name', 'dyad_dset_id',
       'dyad_new_id', 'dyad_name', 'side_a_dset_id', 'side_a_new_id', 'side_a',
       'side_b_dset_id', 'side_b_new_id', 'side_b', 'where_prec',
       'where_coordinates', 'where_description', 'adm_1', 'adm_2', 'latitude',
       'longitude', 'geom_wkt', 'priogrid_gid', 'country', 'country_id',
       'region', 'event_clarity', 'date_prec', 'date_start', 'date_end',
       'best_est', 'COORDS'],
      dtype='object')


In [13]:
variance = np.var(y, axis=0)
variance

8505.836048050072

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# model = ElasticNetCV(cv=2, random_state=0)
model = ElasticNet(random_state=0)
model.fit(X_train, y_train)

print(np.unique(model.coef_))
print(f'score: {model.score(X_test, y_test)}')

[-1.58542042e+01 -6.26873991e+00 -2.36514630e+00 -2.28000950e+00
 -1.11841684e+00 -8.61333411e-01 -8.60583338e-01 -8.37594178e-01
 -6.51264295e-01 -5.73285334e-01 -5.41301519e-01 -2.39470521e-01
 -2.18867259e-01 -1.89350059e-01 -1.80230797e-01 -1.80023300e-01
 -1.37527810e-01 -6.59737544e-02 -5.71641292e-02 -5.70106012e-02
 -1.67423267e-02  0.00000000e+00  1.45227603e-01  1.45227613e-01
  1.88136894e-01  1.88139591e-01  1.93472946e-01  1.93472947e-01
  2.54860936e-01  3.56429895e-01  3.81531691e-01  3.81532035e-01
  3.81532094e-01  3.86106628e-01  3.86106673e-01  3.87541763e-01
  4.11770358e-01  4.11770370e-01  4.11770448e-01  4.11770504e-01
  4.11772353e-01  6.20228557e-01  6.99896776e-01  7.21424353e-01
  8.07168880e-01  8.22273608e-01  8.22274103e-01  8.22274103e-01
  1.16110758e+00  1.24679534e+00  1.27327957e+00  1.27327962e+00
  1.58830905e+00  1.72911750e+00  1.86587254e+00  1.86587397e+00
  1.99408533e+00  1.99408533e+00  1.99408538e+00  1.99408538e+00
  1.99408538e+00  1.99408

In [12]:
y_pred = model.predict(X_test)
yp=y_pred

mse = mean_squared_error(y_test, yp)
print(f"Mean Squared Error: {mse}")

Mean Squared Error: 3048.124686120759
